In [2]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317145 sha256=b91eadc6ba6b297730c22290edf88faf37e660cff01b897ea752b9bd8924d54c
  Stored in directory: /root/.cache/pip/wheels/9f/34/a4/159aa12d0a510d5ff7c8f0220abbea42e5d81ecf588c4fd884
Successfully built pyspark


In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_set, udf,col
from pyspark.sql.types import ArrayType, StringType, IntegerType

# Initialize a Spark session
spark = SparkSession.builder.appName("PCYAlgorithm").getOrCreate()

# Read the data from the transactions.csv file into a PySpark dataframe
data = spark.read.csv("/content/transactions.csv", header=True, inferSchema=True,sep=';')

In [4]:
data = data.select("BillNo", "Itemname") \
    .groupBy("BillNo") \
    .agg(collect_set("Itemname").alias("items")) \
    .select("items")

In [5]:
from itertools import combinations
def generate_candidate_pairs(x,n):
    return combinations(x, n)
support = 30  

In [6]:
data.show(10)

+--------------------+
|               items|
+--------------------+
|[RED WOOLLY HOTTI...|
|[HAND WARMER UNIO...|
|[BLUE COAT RACK P...|
|[MINI JIGSAW SPAC...|
|[EDWARDIAN PARASO...|
|[EDWARDIAN PARASO...|
|[HANGING MEDINA L...|
|[JAM MAKING SET P...|
|[FAIRY TALE COTTA...|
|[GINGHAM HEART  D...|
+--------------------+
only showing top 10 rows



Pass #1

In [7]:
from operator import add
sc = spark.sparkContext
final_counts_rdd = sc.parallelize([])

item_counts = data.rdd.flatMap(lambda x: x.items)\
                        .map(lambda x: (x, 1))\
                        .reduceByKey(add)\
                        .filter(lambda x : x[1]>support)\
                        .collectAsMap()
final_counts_rdd = final_counts_rdd.union(sc.parallelize([item_counts]))

In [8]:
hash_table_size = 1000
def hash_pair(item1, item2):
    return hash(item1 + item2) % hash_table_size

#hasing in pass1 of pcy

In [9]:
hash_pair_counts = data.rdd.flatMap(lambda x: combinations(x.items, 2)).map(lambda x: (hash_pair(x[0],x[1]), 1)).reduceByKey(lambda a, b: a + b).collectAsMap()

In [10]:

candid_pair = data.rdd.flatMap(lambda x: combinations(x.items, 2)).filter(lambda x: hash_pair_counts[hash_pair(x[0],x[1])]>support and all(elem in item_counts for elem in x)
 ).map(lambda x: (x, 1))\
                        .reduceByKey(add)\
                        .filter(lambda x:x[1]>support)\
                       .collectAsMap()
final_counts_rdd = final_counts_rdd.union(sc.parallelize([candid_pair]))
# candid_pair = data.rdd.flatMap(lambda x: combinations(x.items, 3)).filter(lambda x: all(elem in candid_pair for elem in combinations(x, 2))
#  ).map(lambda x: (x, 1))\
#                         .reduceByKey(add)\
#                         .filter(lambda x:x[1]>100)\
#                         .collectAsMap()
        

In [ ]:
n=3
while True:
  candid_pair = data.rdd.flatMap(lambda x: combinations(x.items, n)).filter(lambda x: all(elem in candid_pair for elem in combinations(x, n-1))
 ).map(lambda x: (x, 1))\
                        .reduceByKey(add)\
                        .filter(lambda x:x[1]>support)\
                        .collectAsMap()
  if len(candid_pair)==0:
    print('no frequent_item found for subset more than {} elemnets'.format(n))
    break
  else:
    print(candid_pair)
  n = n+1
  final_counts_rdd = final_counts_rdd.union(sc.parallelize([candid_pair]))


no frea 3


In [11]:
from functools import reduce
combined_dict = final_counts_rdd.reduce(lambda x, y: {**x, **y})
rdd = sc.parallelize([combined_dict])

In [12]:
combined_dict

{'KNITTED UNION FLAG HOT WATER BOTTLE': 209,
 'WHITE HANGING HEART T-LIGHT HOLDER': 1102,
 'WHITE METAL LANTERN': 167,
 'HAND WARMER UNION JACK': 98,
 'YELLOW COAT RACK PARIS FASHION': 58,
 'ALARM CLOCK BAKELIKE PINK': 281,
 'MINI JIGSAW CIRCUS PARADE': 40,
 'POSTAGE': 399,
 'ALARM CLOCK BAKELIKE RED': 410,
 'CIRCUS PARADE LUNCH BOX': 171,
 'PANDA AND BUNNIES STICKER SHEET': 37,
 'LUNCH BOX I LOVE LONDON': 348,
 'CHARLOTTE BAG DOLLY GIRL DESIGN': 146,
 'VINTAGE SEASIDE JIGSAW PUZZLES': 70,
 'SPACEBOY LUNCH BOX': 370,
 'WOOD S/3 CABINET ANT WHITE FINISH': 260,
 'VINTAGE BILLBOARD DRINK ME MUG': 120,
 'WOOD BLACK BOARD ANT WHITE FINISH': 289,
 'HEART OF WICKER LARGE': 381,
 'WHITE LOVEBIRD LANTERN': 87,
 'NATURAL SLATE HEART CHALKBOARD': 587,
 'ENAMEL FIRE BUCKET CREAM': 58,
 'CLASSIC METAL BIRDCAGE PLANT HOLDER': 138,
 'ENAMEL FLOWER JUG CREAM': 273,
 'ORGANISER WOOD ANTIQUE WHITE': 75,
 'TRADITIONAL CHRISTMAS RIBBONS': 69,
 'FAIRY TALE COTTAGE NIGHTLIGHT': 61,
 '5 HOOK HANGER MAGIC TOA

In [ ]:
freq_items_rddr = sc.parallelize([(str(k), v) for k, v in combined_dict.items()])

In [ ]:
freq_items_df = freq_items_rddr.toDF(["items", "support"])

In [ ]:
freq_items_df.collect()[-1]

Row(items="('HAND WARMER UNION JACK', 'HAND WARMER BIRD DESIGN')", support=31)

In [ ]:
freq_items_rddr = sc.parallelize([(str(k), v) for k, v in combined_dict.items()])
freq_items_df = freq_items_rddr.toDF(["items", "support"])

In [ ]:
freq_items_df.show()

+--------------------+-------+
|               items|support|
+--------------------+-------+
|RED WOOLLY HOTTIE...|    106|
|SET 7 BABUSHKA NE...|     55|
|KNITTED UNION FLA...|     83|
|WHITE HANGING HEA...|    144|
| WHITE METAL LANTERN|     34|
|CREAM CUPID HEART...|     36|
|GLASS STAR FROSTE...|     36|
|HAND WARMER UNION...|     89|
|FELTCRAFT PRINCES...|     31|
|POPPY'S PLAYHOUSE...|     47|
|ASSORTED COLOUR B...|     68|
|LOVE BUILDING BLO...|     32|
|POPPY'S PLAYHOUSE...|     47|
|HOME BUILDING BLO...|     47|
|JAM MAKING SET WI...|     61|
|RED TOADSTOOL LED...|     38|
|ROUND SNACK BOXES...|     31|
|SET 2 TEA TOWELS ...|     44|
|ALARM CLOCK BAKEL...|     41|
|             POSTAGE|     38|
+--------------------+-------+
only showing top 20 rows



In [ ]:
subsets_df = (freq_items_df
              .selectExpr("items as left_items", "support as left_support")
              .crossJoin(freq_items_df.selectExpr("items as right_items", "support as right_support"))
              .where("left_items != right_items"))

In [13]:
from itertools import combinations, chain
def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(1,len(s)))
rdd = sc.parallelize(list(combined_dict.items()))

In [22]:
for i in combined_dict:
    if type(i)==tuple:
        item_power = powerset(i)
        for t in item_power:
            p = set(i)-set(t)
            right_support = combined_dict[i]
            if len(tuple(p))==1:
                left_support = combined_dict[tuple(p)[0]]
            else:
                left_support = combined_dict[tuple(p)]
            confidece = right_support/left_support
            if confidece>0.5:
                print("found {} -> {}".format(p,t))

found {'RED COAT RACK PARIS FASHION'} -> ('BLUE COAT RACK PARIS FASHION',)
found {'BLUE COAT RACK PARIS FASHION'} -> ('RED COAT RACK PARIS FASHION',)
found {'ALARM CLOCK BAKELIKE PINK'} -> ('ALARM CLOCK BAKELIKE RED',)
found {'BLUE DINER WALL CLOCK'} -> ('IVORY DINER WALL CLOCK',)
found {'LARGE POPCORN HOLDER'} -> ('SMALL POPCORN HOLDER',)
found {'SET/6 RED SPOTTY PAPER PLATES'} -> ('SET/6 RED SPOTTY PAPER CUPS',)
found {'SET/6 RED SPOTTY PAPER CUPS'} -> ('SET/6 RED SPOTTY PAPER PLATES',)
found {'60 TEATIME FAIRY CAKE CASES'} -> ('PACK OF 72 RETROSPOT CAKE CASES',)
found {'KITCHEN METAL SIGN'} -> ('BATHROOM METAL SIGN',)
found {'TOILET METAL SIGN'} -> ('BATHROOM METAL SIGN',)
found {'CANDLEHOLDER PINK HANGING HEART'} -> ('WHITE HANGING HEART T-LIGHT HOLDER',)
found {'SET OF 4 ENGLISH ROSE COASTERS'} -> ('SET OF 4 ENGLISH ROSE PLACEMATS',)
found {'SET OF 4 ENGLISH ROSE PLACEMATS'} -> ('SET OF 4 ENGLISH ROSE COASTERS',)
found {'PAPER CHAIN KIT VINTAGE CHRISTMAS'} -> ("PAPER CHAIN KIT 50'